In [1]:
import masterjdv

Estimation des temps de restitution pour 10 x 10 100 x 100 et 1000 x 1000

In [2]:
grid = masterjdv.init_grid(10)
%timeit masterjdv.evolution1(grid)

870 µs ± 5.79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
grid = masterjdv.init_grid(100)
%timeit masterjdv.evolution1(grid)

94.1 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
grid = masterjdv.init_grid(1000)
%timeit masterjdv.evolution1(grid)

9.99 s ± 881 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


On constate un temps d'exécution qui croit linéairement en fonction de la taille du problème (nombre de cellule  x100 à chaque fois) ce qui est cohérent avec notre algorithme qui applique un même calcul pour chaque cellule de la grille.

# Profilage

Définition d'une grille 2000 x 2000

In [5]:
grid = masterjdv.init_grid(2000)

Profil de evolution1

In [6]:
import cProfile
cProfile.run("masterjdv.evolution1(grid)")

         32000007 function calls in 51.414 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.065    0.065   51.414   51.414 <string>:1(<module>)
        1    5.149    5.149   51.349   51.349 masterjdv.py:25(evolution1)
        1    0.004    0.004    0.004    0.004 masterjdv.py:28(<listcomp>)
  4000000   40.896    0.000   45.542    0.000 masterjdv.py:6(get_nbneigh)
        1    0.000    0.000   51.414   51.414 {built-in method builtins.exec}
  8000002    0.931    0.000    0.931    0.000 {built-in method builtins.len}
  8000000    2.002    0.000    2.002    0.000 {built-in method builtins.max}
  8000000    1.713    0.000    1.713    0.000 {built-in method builtins.min}
  4000000    0.654    0.000    0.654    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# Inlining
Définition de la fonction evolution1_corr dans laquelle on intègre le calcul de nb_neigh (inlining)

In [7]:
def evolution1_corr(grid):
    irange=grid.shape[0]
    jrange=grid.shape[1]
    res_grid=[[] for i in range(irange)]
    for j in range(jrange):
        for i in range(irange):
            # by default cell_state is dead
            cell_state=0
            # loop over neighs to count living cells
            biinf=max(0,i-1)
            bisup=min(irange-1,i+1)
            bjinf=max(0,j-1)
            bjsup=min(jrange-1,j+1)
            nb_neigh=0
            # loop over neighs to count living cells
            for ii in range(biinf,bisup+1):
                for jj in range(bjinf,bjsup+1):
                    if grid[ii][jj] == 1:
                        nb_neigh=nb_neigh+1
            # in my count I count current cell itself substract the value
            nb_neigh=nb_neigh-grid[i][j]
            # if 2 neighbors cell state isn't modified
            if nb_neigh==2:
                cell_state=grid[i][j]
            # if 3 neighbors cell state is alive
            elif nb_neigh==3:
                cell_state=1
            # in other case cell state is dead (default state)
            res_grid[i].append(cell_state)
    return res_grid

Profil de evolution1_corr

In [8]:
cProfile.run("evolution1_corr(grid)")

         20000005 function calls in 44.978 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   40.917   40.917   44.904   44.904 <ipython-input-7-4661d96cf506>:1(evolution1_corr)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-4661d96cf506>:4(<listcomp>)
        1    0.075    0.075   44.978   44.978 <string>:1(<module>)
        1    0.000    0.000   44.978   44.978 {built-in method builtins.exec}
  8000000    1.902    0.000    1.902    0.000 {built-in method builtins.max}
  8000000    1.626    0.000    1.626    0.000 {built-in method builtins.min}
  4000000    0.458    0.000    0.458    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Résultats
On constate une réduction du nombre d'appels de fonction :
- 32000007 pour la 1ère version contre 20000005 pour la 2nde
- en particulier 8000002 appel à la méthode len

# Structures de données
Importation du package numpy et définition de la fonction evolution1_ndarray dans laquelle on utilise des structures de données définies pour le traitement des tableaux : ndarray

In [9]:
import numpy as np

In [10]:
def evolution1_ndarray(grid):
    irange=grid.shape[0]
    jrange=grid.shape[1]
    res_grid=np.zeros((irange,jrange))
    for i in range(irange):
        for j in range(jrange):
            # by default cell_state is dead
            cell_state=0
            # loop over neighs to count living cells
            biinf=max(0,i-1)
            bisup=min(irange-1,i+1)
            bjinf=max(0,j-1)
            bjsup=min(jrange-1,j+1)
            nb_neigh=0
            # loop over neighs to count living cells
            for ii in range(biinf,bisup+1):
                for jj in range(bjinf,bjsup+1):
                    if grid[ii][jj] == 1:
                        nb_neigh=nb_neigh+1
            # in my count I count current cell itself substract the value
            nb_neigh=nb_neigh-grid[i][j]
            # if 2 neighbors cell state isn't modified
            if nb_neigh==2:
                cell_state=grid[i][j]
            # if 3 neighbors cell state is alive
            elif nb_neigh==3:
                cell_state=1
            # in other case cell state is dead (default state)
            res_grid[i,j]=cell_state
    return res_grid

Profil de evolution1_ndarray

In [11]:
cProfile.run("evolution1_ndarray(grid)")

         16000005 function calls in 49.257 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   45.405   45.405   49.254   49.254 <ipython-input-10-c0654f4ebe16>:1(evolution1_ndarray)
        1    0.003    0.003   49.256   49.256 <string>:1(<module>)
        1    0.000    0.000   49.257   49.257 {built-in method builtins.exec}
  8000000    2.095    0.000    2.095    0.000 {built-in method builtins.max}
  8000000    1.755    0.000    1.755    0.000 {built-in method builtins.min}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Estimation des temps d'exécution pour chacune des trois fonctions

In [13]:
%timeit masterjdv.evolution1(grid)

43.7 s ± 3.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit evolution1_corr(grid)

42.6 s ± 1.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit evolution1_ndarray(grid)

39.3 s ± 2.74 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Résultats
On constate un gain de performance relativement faible : 2% avec evolution1_corr 7% (resp. 10%) avec evolution1_ndarray par rapport à evolution1_corr (resp. evolution1)